In [231]:
import networkx as nx
import networkx.algorithms.community.label_propagation as label_prop
import networkx.algorithms.community.modularity_max as modularity
import networkx.algorithms.components.connected as components
import networkx as nx
from collections import defaultdict

CLUSTERING_FN = {
    1: label_prop.label_propagation_communities,
    2: modularity.greedy_modularity_communities,
    3: components.connected_components
}
def ego_splitting(G,lc,gc):
    S_prime = set()
    nn=1
    for u in G.nodes():
        ego_net = G.subgraph(nx.ego_graph(G, u))
        N_u = list(ego_net.nodes())
        N_u.remove(u)
        partitions = A_local(ego_net,lc)
        personas = set()
        for i, partition in enumerate(partitions):
            for node in partition:
                personas.add((u, i, node))
        G_prime = nx.Graph()
        G_prime.add_nodes_from(personas)
        for v in N_u:
            partition_v = A_local(G.subgraph(nx.ego_graph(G, v, radius=2)),lc)
            for j, cluster_j in enumerate(partition_v):
                for w in cluster_j:
                    G_prime.add_edge((u, N_u.index(v), v), (v, j, w))
        clusters = A_global(G_prime,gc)
        # print(f"node number {nn}")
        # nn+=1
        for cluster in clusters:
            nodes = set()
            for persona in cluster:
                nodes.add(persona[2])
            #print("Inside adding nodes")
            S_prime.add(frozenset(nodes))
    return S_prime


 

# Define the two clustering algorithms
def A_local(graph,k):
  return CLUSTERING_FN[k](graph)
  # nx.algorithms.community.label_propagation.label_propagation_communities(graph)

def A_global(graph,k):
  return CLUSTERING_FN[k](graph)

lc=int(input("Select the clustering algorithm for local clustering from the options \n1.Label Propogation\n 2.Modularity Max\n,3.Connected Components\n"))
gc=int(input("Select the clustering algorithm for Global clustering from the options \n1.Label Propogation\n 2.Modularity Max\n,3.Connected Components\n"))

import networkx as nx

n = 500  # number of nodes
tau1 = 2.5  # power-law exponent for the degree distribution of the communities
tau2 = 1.5  # power-law exponent for the degree distribution within the communities
mu = 0.1  # fraction of edges that are between communities
m = 2000  # total number of edges

# Compute the average degree based on the number of edges
k = int(2 * m / n)

# Generate the LFR benchmark graph
G = nx.LFR_benchmark_graph(n, tau1, tau2, mu, average_degree=k, min_community=30, seed=0)

# Get the ground truth clusters
ground_truth = []
for node, community in nx.get_node_attributes(G, "community").items():
    # print(community)
    ground_truth.append(frozenset(community))



# Print the number of nodes and edges in the graph
print('Number of nodes:', G.number_of_nodes())
print('Number of edges:', G.number_of_edges())


S_prime = ego_splitting(G,lc,gc)

print(S_prime)

Number of nodes: 500
Number of edges: 2160
{frozenset({129, 8, 264, 269, 272, 25, 282, 409, 33, 291, 164, 165, 41, 171, 49, 50, 183, 314, 61, 321, 66, 322, 452, 325, 326, 323, 202, 203, 330, 462, 211, 343, 219, 224, 360, 489, 235, 493, 494, 124, 241, 242, 243, 244, 252}), frozenset({2, 264, 11, 269, 15, 274, 20, 280, 283, 29, 294, 47, 309, 74, 75, 330, 77, 331, 333, 80, 336, 344, 89, 345, 91, 346, 106, 362, 363, 366, 367, 112, 368, 369, 115, 370, 371, 373, 120, 377, 380, 381, 382, 127, 131, 388, 389, 134, 390, 136, 137, 391, 139, 394, 396, 397, 145, 146, 402, 150, 406, 407, 153, 408, 409, 156, 158, 159, 414, 161, 418, 163, 421, 422, 167, 423, 169, 425, 427, 173, 174, 431, 180, 187, 443, 189, 444, 449, 194, 450, 454, 458, 474, 221, 479, 481, 482, 483, 484, 488, 494, 499, 245, 246, 248}), frozenset({86}), frozenset({257, 3, 391, 392, 9, 137, 267, 396, 13, 142, 271, 10, 404, 283, 158, 161, 290, 35, 168, 301, 302, 53, 182, 440, 320, 65, 196, 69, 454, 202, 335, 212, 468, 214, 90, 218, 93, 3

In [232]:
def demon_algorithm(G):
    # Initialize empty set of communities
    C = set()

    # Iterate over all nodes in the graph
    for v in G.nodes():
        # Remove the ego network of the node
        e = nx.ego_graph(G, v, radius=1)
        e.remove_node(v)

        # Run label propagation algorithm on the ego network
        labels = nx.algorithms.community.label_propagation.label_propagation_communities(e)

        # Add each community to C and include the current node in it
        for label in labels:
            label.add(v)
            C.add(frozenset(label))

    return C
Demon_clusters=demon_algorithm(G)

In [233]:
def calc_f1(c1,c2):
    precision=len(frozenset.intersection(c1,c2))/len(c1)
    recall=len(frozenset.intersection(c1,c2))/len(c2)
    try:
        return 2*((precision*recall)/(precision+recall))
    except:
        return 0
def calc_prec(groundtruth,S_prime):
    k=0
    #c inter c dash/c dash
    for i in S_prime :
        maxi=-999999
        for j in groundtruth:
            maxi=max(maxi,calc_f1(i,j))
        k+=maxi
    return k/len(S_prime)

In [235]:

ground_truth=set(ground_truth)
print("f1 score using egosplitting is  is {0:.6f}".format(calc_prec(S_prime,ground_truth)))
print("f1 score using Demon algorithm is {0:.6f}".format(calc_prec(Demon_clusters,ground_truth)))

f1 score using egosplitting is  is 0.826704
f1 score using Demon algorithm is 0.486859
